In [2]:
import os
import io
import glob
import pandas as pd
import numpy as np
import tensorflow as tf # ignore horrendous TF1.XX dep errors

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

In [3]:
# local path to github repository, change as you see fit
WORKING_DIR = '/Users/ebyler/gh/dice_detection'

# Generating TFRecord files

NOTE: The following code is from Dat Tran's raccoon object identification project (https://github.com/datitran/raccoon_dataset) `generate_tfrecord.py`, which was written for tensorflow 1.XX. I had originally intended to do this project in tensorflow 2.0, and wanted to translate Dat Tran's code to work for tf 2.0.

Spoiler alert - you can! But this may not be a rabbit hole you want to go down. At the time of this project, none of the extensive object detection scripts are tf 2.0 compatible, and it will probably cause you some serious headaches down the line. An upgrade will probably happen SOON$^{TM}$, but until then, I have reverted back to tf 1.14.
(Read more about it here: https://github.com/tensorflow/models/issues/6423)


### TF 2.0 changes:
- Remove flag code (I was too lazy to try and add new flag system, using e.g. abd).
- L32: replace `tf.file.GFile` with `tf.compat.v1.gfile.GFile`.
- In writing stage: replace `tf.python_io` with `tf.compat.v1.python_io`.

In [4]:
# Replace this with label map
# Note -- per TFRecord documentation,
# the label IDs must start with id = 1, not 0!

# also note -- even though I have numerical classes,
# the TFRecord code expects strings. 

def class_text_to_int(row_label):
    if row_label == 'one':
        return 1
    elif row_label == 'two':
        return 2
    elif row_label == 'three':
        return 3
    elif row_label == 'four':
        return 4
    elif row_label == 'five':
        return 5
    elif row_label == 'six':
        return 6
    else:
        None


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


# Write training TFRecord

In [5]:
csv_input = WORKING_DIR+'/data/ImageSets/Main/train_labels.csv'
image_dir = WORKING_DIR+'/data/JPEGImages'

SAVE_DIR = WORKING_DIR+'/tfrecords'
output_path = SAVE_DIR+'/train.record'

In [6]:
# the following line needs to change for TF 2.0 
# >> tf.compat.v1.python_io
writer = tf.python_io.TFRecordWriter(output_path)
path = os.path.join(image_dir)
examples = pd.read_csv(csv_input)
grouped = split(examples, 'filename')
for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())
    
writer.close()
output_path = os.path.join(os.getcwd(), output_path)
print('Successfully created the TFRecords: {}'.format(output_path))

Successfully created the TFRecords: /Users/ebyler/gh/dice_detection/tfrecords/train.record


# Write test TFRecord

In [7]:
csv_input = WORKING_DIR+'/data/ImageSets/Main/test_labels.csv'
image_dir = WORKING_DIR+'/data/JPEGImages'

SAVE_DIR = WORKING_DIR+'/tfrecords'
output_path = SAVE_DIR+'/test.record'

In [8]:
writer = tf.python_io.TFRecordWriter(output_path)
path = os.path.join(image_dir)
examples = pd.read_csv(csv_input)
grouped = split(examples, 'filename')
for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())
    
writer.close()
output_path = os.path.join(os.getcwd(), output_path)
print('Successfully created the TFRecords: {}'.format(output_path))

Successfully created the TFRecords: /Users/ebyler/gh/dice_detection/tfrecords/test.record
